In [2]:
import cv2
import numpy as np
from PIL import Image
import os

In [3]:
#Open Camera
cam = cv2.VideoCapture(0)

# Camera dimensions
cam.set(3, 720) 
cam.set(4, 640) 

face_identity = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Enter Unique ID
user_id = input('\n Enter Used ID =  ')

# Initialize individual sampling face count
pics = 0
while(True):
    ret, img = cam.read()
    img = cv2.flip(img, 1) # flip video image vertically
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_identity.detectMultiScale(gray, 1.2, 5)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)     
        pics += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("dataset/Face." + str(user_id) + '.' + str(pics) + ".jpg", gray[y:y+h,x:x+w])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff 
    if k == 27:
        break
    elif pics >= 50:
         break
# Close windows
cam.release()
cv2.destroyAllWindows()


 Enter Used ID =  1


## Train


In [4]:
# Location where pics saved
img_path = 'dataset'

# Recognize and identify
recognizer = cv2.face.LBPHFaceRecognizer_create()

identifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");

# Detect each user face and assign id the
def face_detect(img_path):

    locs = [os.path.join(img_path,f) for f in os.listdir(img_path)]     
    Samples=[]
    User_ids = []

    for loc in locs:

        PIL_img = Image.open(loc).convert('L') # convert it to grayscale
        img_numpy = np.array(PIL_img,'uint8')

        id = int(os.path.split(loc)[-1].split(".")[1])
        faces = identifier.detectMultiScale(img_numpy)

        for (x,y,w,h) in faces:
            Samples.append(img_numpy[y:y+h,x:x+w])
            User_ids.append(id)

    return Samples,User_ids

print ("\n Training User faces. Please Wait ...")
faces,User_ids = face_detect(img_path)
recognizer.train(faces, np.array(User_ids))

# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainer.yml') 

# Number of user face id trained.
print("\n {0} faces trained.".format(len(np.unique(User_ids))))


 Training User faces. Please Wait ...

 1 faces trained.


## Test

In [ ]:

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);
font = cv2.FONT_HERSHEY_SIMPLEX

User_ids = 0
# names related to ids: 
person = ['None', 'Rahul','Sandeep','Adithya',"Ashok"] 
# Open Camera and 
cam = cv2.VideoCapture(0)
cam.set(3, 720) # set video widht
cam.set(4, 640) # set video height
# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
while True:
    ret, img =cam.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        User_ids, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        if (confidence > 30 and confidence < 100):
            User_ids = person[User_ids]
            confidence = "  {0}%".format(round(confidence))
        else:
            User_ids = "unknown"
            confidence = "  {0}%".format(round(confidence))
        
        cv2.putText(img, str(User_ids), (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
    
    cv2.imshow('camera',img) 
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()
